In [2]:
import os
import numpy as np
from Bio import SeqIO
from tqdm import tqdm
import matplotlib.pyplot as plt
from Bio.Seq import Seq
import pandas as pd

In [17]:
#create list of fastq files we will want to merge
fastq_list = []
f = 'fastq'

for i in os.listdir():
    if "fastq" in i:
        fastq_list.append(i)
             
fastq_list.sort()
print(fastq_list)
        

['S18_L001_R1_001.fastq', 'S18_L001_R2_001.fastq', 'S19_L001_R1_001.fastq', 'S19_L001_R2_001.fastq', 'S20_L001_R1_001.fastq', 'S20_L001_R2_001.fastq', 'S21_L001_R1_001.fastq', 'S21_L001_R2_001.fastq', 'S22_L001_R1_001.fastq', 'S22_L001_R2_001.fastq', 'S23_L001_R1_001.fastq', 'S23_L001_R2_001.fastq', 'run1_S1_L001_R1_001.fastq', 'run1_S1_L001_R2_001.fastq', 'run1_S2_L001_R1_001.fastq', 'run1_S2_L001_R2_001.fastq', 'run1_S3_L001_R1_001.fastq', 'run1_S3_L001_R2_001.fastq', 'run1_S4_L001_R1_001.fastq', 'run1_S4_L001_R2_001.fastq', 'run1_S5_L001_R1_001.fastq', 'run1_S5_L001_R2_001.fastq', 'run1_S6_L001_R1_001.fastq', 'run1_S6_L001_R2_001.fastq', 'run2_S10_L001_R1_001.fastq', 'run2_S10_L001_R2_001.fastq', 'run2_S11_L001_R1_001.fastq', 'run2_S11_L001_R2_001.fastq', 'run2_S7_L001_R1_001.fastq', 'run2_S7_L001_R2_001.fastq', 'run2_S8_L001_R1_001.fastq', 'run2_S8_L001_R2_001.fastq', 'run2_S9_L001_R1_001.fastq', 'run2_S9_L001_R2_001.fastq', 'run3_S12_L001_R1_001.fastq', 'run3_S12_L001_R2_001.fastq

In [18]:
#iterate through fastq list and merge paired-reads with bbmerge

index = 0
merged_list = []


while index < len(fastq_list):
    sample = fastq_list[index].split('_')[len(fastq_list[index].split('_'))-4]
    os.system(f'. ./bbmap/bbmerge.sh in1={fastq_list[index]} in2={fastq_list[index+1]} out=merged_{sample}.fastq outu=unmerged_{sample}.fastq')
    merged_list.append('merged_' + sample + '.fastq')
    index += 2
    


In [19]:
#if merged files already exist, run to remake list of merged files
merged_list = []
for i in os.listdir():
    if "merged" in i and "unmerged" not in i:
        merged_list.append(i)
             
merged_list.sort()
print(merged_list)
        

['merged_S1.fastq', 'merged_S10.fastq', 'merged_S11.fastq', 'merged_S12.fastq', 'merged_S13.fastq', 'merged_S14.fastq', 'merged_S15.fastq', 'merged_S16.fastq', 'merged_S17.fastq', 'merged_S18.fastq', 'merged_S19.fastq', 'merged_S2.fastq', 'merged_S20.fastq', 'merged_S21.fastq', 'merged_S22.fastq', 'merged_S23.fastq', 'merged_S3.fastq', 'merged_S4.fastq', 'merged_S5.fastq', 'merged_S6.fastq', 'merged_S7.fastq', 'merged_S8.fastq', 'merged_S9.fastq']


In [21]:
forward_primer = "GAACGTGAGTTTGTGGCG" #shorter version inclusive of both
reverse_primer = "CCCCAGTAATCGTAATC"

"""set up empty lists to count various occurances"""
i = 0
read_num = [0]*len(merged_list) #total reads per sample
counts_forward = [0]*len(merged_list) #num reads with forward primer(s)
counts_reverse = [0]*len(merged_list) #num reads with reverse primer

nok = [0]*len(merged_list) #num reads with specifically the no K forward primer

low_phred = [0]*len(merged_list) #reads tossed for low phred score
not_found = [0]*len(merged_list) #num reads where primers aren't found

seq_df = pd.DataFrame()
well_dict = {} #wells = keys, where values = dict of read seq + # of appearances
not_found_list = []

def trim(sequence, primer):
    start = str(record.seq).find(primer)
    return str(record.seq)[start:start+208]

"""now let's iterate through the merged files and trim reads"""
for file in merged_list: #iterate through all the merged files
    
    prefix = file.split('.')[0]
    well_num = prefix.split('_')[1] #determine sample number, ex: S1
    
    well_dict[well_num] = {} #add well to overall dictionary 
    for record in tqdm(SeqIO.parse(file, 'fastq')): #iterate through all the reads
        read_num[i] += 1
        if np.mean(record.letter_annotations["phred_quality"]) >= 20: #only enter if high quality read
            
            sequence = str(record.seq)
            
            #forwardize all sequences
            if reverse_primer in sequence:
                sequence = str(Seq(sequence).reverse_complement())
                counts_reverse[i] += 1
            
            if forward_primer in sequence:
                counts_forward[i] += 1
                start = sequence.find(forward_primer)
                sequence = sequence[start:start+208]
                
                if sequence in well_dict[well_num]:
                    well_dict[well_num][sequence] += 1
                else:
                    well_dict[well_num][sequence] = 1
            else:
                not_found[i] += 1
                not_found_list.append(sequence)
                
            
        else:
            low_phred[i] += 1    

    i += 1        
                


224050it [00:09, 24515.24it/s]
236882it [00:09, 25516.31it/s]
183693it [00:07, 25012.37it/s]
439883it [00:19, 23017.59it/s]
252151it [00:09, 25259.47it/s]
259627it [00:10, 24975.61it/s]
234862it [00:09, 25500.54it/s]
198711it [00:08, 23758.11it/s]
303823it [00:11, 25508.04it/s]
208002it [00:09, 21572.91it/s]
181058it [00:07, 25457.26it/s]
386789it [00:15, 25134.17it/s]
354372it [00:14, 25174.24it/s]
396846it [00:18, 21720.63it/s]
320414it [00:12, 25322.33it/s]
414853it [00:16, 24769.44it/s]
271549it [00:10, 25013.92it/s]
209420it [00:08, 24429.31it/s]
202062it [00:08, 25161.44it/s]
227975it [00:10, 21811.05it/s]
389490it [00:15, 24878.66it/s]
413532it [00:17, 23945.90it/s]
255802it [00:10, 24546.88it/s]


In [22]:
#convert to pandas dataframe and concatinate

big_df = pd.DataFrame()

working_df = pd.DataFrame(list(well_dict['S1'].items()), columns=['seq','S1'])
working_df = working_df.set_index('seq')

for sample in well_dict.keys():
    working_df = pd.DataFrame(list(well_dict[sample].items()), columns=['seq',sample])
    working_df = working_df.set_index('seq')
    conc_df = pd.concat([big_df, working_df], axis = 1 )
    big_df = conc_df
    
   


                                                    S1
seq                                                   
GAACGTGAGTTTGTGGCGGCCATTAGCTTCTCTGGGGCCATCACCTA...   2
GAACGTGAGTTTGTGGCGACCATTGGCGTCTCTGGGGCCAACACCTA...   2
GAACGTGAGTTTGTGGCGAGCATTACCAACTCTGGGGGCAGCACCTA...   2
GAACGTGAGTTTGTGGCGGGCATTAACTACTCTGGGGGCAACACCTA...   1
GAACGTGAGTTTGTGGCGAGCATTGACTCCTCTGGGGGCAACACCTA...   1
...                                                 ..
GAACGTGAGTTTGTGGCGGGCATTACCAACTCTGGGACCATCACCAA...   1
GAACGTGAGTTTGTGGCGGGCATTGCCCTCTCTGGGAGCAACACCTA...   1
GAACGTGAGTTTGTGGCGGCCATTACCCTCTCTGGGGCCAACACCTA...   1
GAACGTGAGTTTGTGGCGGCCATTGACGTCTCTGGGACCATCACCTA...   1
GAACGTGAGTTTGTGGCGGCCATTAACGTCTCTGGGGGCATCACCTA...   1

[175455 rows x 1 columns]


In [24]:
#lets try to merge the dataframes together instead to only keep sequences that appear in all


run1 = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
run2 = ['S7', 'S8', 'S9', 'S10', 'S11']
run3 = ['S12', 'S13', 'S14', 'S15', 'S16', 'S17']

i = 1
for sample in run1:
    if i == 1:
        merged_df = pd.DataFrame(list(well_dict[sample].items()), columns=['seq',sample])
        merged_df = merged_df.set_index('seq')
    else:
        working_df = pd.DataFrame(list(well_dict[sample].items()), columns=['seq',sample])
        working_df = working_df.set_index('seq')
        df = pd.merge(merged_df, working_df, left_index=True, right_index=True)
        #df = pd.concat([merged_df, working_df], axis = 1 )
        merged_df = df
    i += 1
    
print(merged_df)
merged_df.to_csv('col1_conc.csv')   
   

                                                     S1     S2      S3     S4  \
seq                                                                             
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...  562  47296  180150  26785   
GAACGTGAGTTTGTGGCGGCCATTGCCCTCTCTGGGAGCATCACCTA...    1     13       3     12   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...    2    732     248    100   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...    1     30      38     43   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...    1      4      15      2   
...                                                 ...    ...     ...    ...   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCCACCTA...    1     14      68     13   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...    1    930     552    614   
GAACGTGAGTTTGTGGCGGCCATTGGCTTCTCTGGGAGCATCACCTA...    1     24       3      2   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...    1      8      26      5   
GAACGTGAGTTTGTGGCGGCCATTAGCT

In [23]:
i = 1
for sample in run2:
    if i == 1:
        merged_df = pd.DataFrame(list(well_dict[sample].items()), columns=['seq',sample])
        merged_df = merged_df.set_index('seq')
    else:
        working_df = pd.DataFrame(list(well_dict[sample].items()), columns=['seq',sample])
        working_df = working_df.set_index('seq')
        df = pd.merge(merged_df, working_df, left_index=True, right_index=True)
        merged_df = df
    i += 1
    
print(merged_df)
#merged_df.to_csv('col2_conc.csv') #remove comment to export to CVS

                                                      S7    S6   S23_x   S17  \
seq                                                                            
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...  1994  2057  106841  1770   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...     1     1      48     2   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...     2     1      49     2   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...    18    38     240    29   
GAACGTGAGTTTGTGGCGACTATTGTTACGTCTGGCGGCGGTACCGG...     2    49       1    20   
...                                                  ...   ...     ...   ...   
GAACGTGAGTTTGTGGCGACTATTGATTCGTCTGGCGGCAGTACCGG...     2    42       1    11   
GAACGTGAGTTTGTGGCGACTATTTTTGCGTCTGGCGGCCGTACCGG...     1    66       1   109   
GAACGTGAGTTTGTGGCGACTATTCATACGTCTGGCGGCAGTACCGG...     1    70       1    27   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCCACCTA...     1     1      42     1   
GAACGTGAGTTTGTGGCGACTATTTATACGTCTGGCGGCC

In [25]:
i = 1
for sample in run3:
    if i == 1:
        merged_df = pd.DataFrame(list(well_dict[sample].items()), columns=['seq',sample])
        merged_df = merged_df.set_index('seq')
    else:
        working_df = pd.DataFrame(list(well_dict[sample].items()), columns=['seq',sample])
        working_df = working_df.set_index('seq')
        df = pd.merge(merged_df, working_df, left_index=True, right_index=True)
        merged_df = df
    i += 1
    
print(merged_df)
#merged_df.to_csv('col3_conc.csv') #remove comment to export to CVS

                                                    S12  S13  S14   S15   S16  \
seq                                                                             
GAACGTGAGTTTGTGGCGACTATTGATCCGTCTGGCGGCATTACCGG...   79    4    1    37    14   
GAACGTGAGTTTGTGGCGACTATTGGTCCGTCTGGCGGCATTACCGG...  346   35   59  2306  9719   
GAACGTGAGTTTGTGGCGACTATTAGTCCGTCTGGCGGCTTTACCGG...   76    1    3    49    49   
GAACGTGAGTTTGTGGCGACTATTAGTCCGTCTGGCGGCTGTACCGG...  151    5    3    47    36   
GAACGTGAGTTTGTGGCGACTATTATTACGTCTGGCGGCGGTACCGG...  198    2    1     8     8   
...                                                 ...  ...  ...   ...   ...   
GAACGTGAGTTTGTGGCGACCATTGCCATCTCTGGGAGCATCACCTA...    1    2   68     1     2   
GAACGTGAGTTTGTGGCGACCATTGCCGTCTCTGGGACCATCACCTA...    1    1   37     2     1   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...    2    4  121    12     8   
GAACGTGAGTTTGTGGCGACTATTTCCTGGTCTGGGGGGTCTACCTA...    1    1   89     3     5   
GAACGTGAGTTTGTGGCGACTATTTCCT

In [28]:
print('read_num: ' + str(read_num))
print('forward counts: ' + str(counts_forward))
print('reverse counts: ' + str(counts_reverse))
print('phred score too low: ' + str(low_phred))
print('primers not found: ' + str(not_found))

read_num: [224050, 236882, 183693, 439883, 252151, 259627, 234862, 198711, 303823, 208002, 181058, 386789, 354372, 396846, 320414, 414853, 271549, 209420, 202062, 227975, 389490, 413532, 255802]
forward counts: [212067, 220135, 172127, 331766, 240571, 247932, 218379, 183375, 277237, 196325, 170286, 272247, 230500, 292257, 224596, 294355, 257991, 195954, 187150, 207508, 283758, 292970, 239943]
reverse counts: [93669, 106467, 81945, 143177, 101078, 122000, 102236, 86816, 131903, 87750, 76571, 112853, 92236, 117948, 102835, 125451, 127014, 93233, 85573, 96470, 113897, 132083, 113119]
phred score too low: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
primers not found: [11983, 16747, 11566, 108117, 11580, 11695, 16483, 15336, 26586, 11677, 10772, 114542, 123872, 104589, 95818, 120498, 13558, 13466, 14912, 20467, 105732, 120562, 15859]
